In [ ]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 8, 3, padding=0) 
        self.bn1 = nn.BatchNorm2d(8)
        self.drop1 = nn.Dropout(0.05)
        
        self.conv2 = nn.Conv2d(8, 32, 3, padding=0)
        self.bn2 = nn.BatchNorm2d(32)
        self.drop2 = nn.Dropout(0.05)
        
        self.conv3 = nn.Conv2d(32, 8, 1, padding=0)
        self.pool3 = nn.MaxPool2d(2, 2)

        self.conv4 = nn.Conv2d(8, 32, 3, padding=0)
        self.bn4 = nn.BatchNorm2d(32)
        self.drop4 = nn.Dropout(0.05)
        
        self.conv5 = nn.Conv2d(32, 32, 3, padding=0)
        self.bn5 = nn.BatchNorm2d(32)
        self.drop5 = nn.Dropout(0.05)
        
        self.conv6 = nn.Conv2d(32, 16, 1, padding=0)
        self.pool6 = nn.MaxPool2d(2, 2)

        self.conv7 = nn.Conv2d(16, 32, 3, padding=0)
        
        self.pool8 = nn.AvgPool2d(2,2)
        
        self.fc1 = nn.Linear(32, 10)
        
    def forward(self, x):
        x = self.drop1(self.bn1(F.relu(self.conv1(x))))
        x = self.drop2(self.bn2(F.relu(self.conv2(x))))
        x = self.pool3(self.conv3(x))
        x = self.drop4(self.bn4(F.relu(self.conv4(x))))
        x = self.drop5(self.bn5(F.relu(self.conv5(x))))
        x = self.pool6(self.conv6(x))
        x = F.relu(self.conv7(x))
        x = self.pool8(x)
        
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        return F.log_softmax(x, dim=1)

In [ ]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 26, 26]              80
       BatchNorm2d-2            [-1, 8, 26, 26]              16
           Dropout-3            [-1, 8, 26, 26]               0
            Conv2d-4           [-1, 32, 24, 24]           2,336
       BatchNorm2d-5           [-1, 32, 24, 24]              64
           Dropout-6           [-1, 32, 24, 24]               0
            Conv2d-7            [-1, 8, 24, 24]             264
         MaxPool2d-8            [-1, 8, 12, 12]               0
            Conv2d-9           [-1, 32, 10, 10]           2,336
      BatchNorm2d-10           [-1, 32, 10, 10]              64
          Dropout-11           [-1, 32, 10, 10]               0
           Conv2d-12             [-1, 32, 8, 8]           9,248
    

In [ ]:


torch.manual_seed(1)
batch_size = 256

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.RandomApply([transforms.CenterCrop(22), ], p=0.1),
                        transforms.Resize((28, 28)),
                        transforms.RandomRotation((-15., 15.), fill=0),
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


100%|██████████| 9912422/9912422 [00:00<00:00, 300987549.82it/s]

Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 108350352.26it/s]


Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 201037510.44it/s]


Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 22281320.20it/s]

Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw



In [ ]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    train_loss = 0
    correct_train = 0
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        
        pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
        correct_train += pred.eq(target.view_as(pred)).sum().item()
        
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx} Train_accuracy = {100. * correct_train/len(train_loader.dataset)}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.3f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

# New Section

In [ ]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 21):
    print('epoch : ',epoch)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

epoch :  1


loss=0.13605201244354248 batch_id=234 Train_accuracy = 77.14: 100%|██████████| 235/235 [00:27<00:00,  8.49it/s]



Test set: Average loss: 0.0903, Accuracy: 9710/10000 (97.100%)

epoch :  2


loss=0.19593112170696259 batch_id=234 Train_accuracy = 96.38: 100%|██████████| 235/235 [00:28<00:00,  8.38it/s]



Test set: Average loss: 0.0487, Accuracy: 9847/10000 (98.470%)

epoch :  3


...


Test set: Average loss: 0.0196, Accuracy: 9930/10000 (99.300%)

epoch :  19


loss=0.002570175798609853 batch_id=234 Train_accuracy = 98.98833333333333: 100%|██████████| 235/235 [00:28<00:00,  8.11it/s]



Test set: Average loss: 0.0180, Accuracy: 9943/10000 (99.430%)

epoch :  20


loss=0.02766178548336029 batch_id=234 Train_accuracy = 99.01166666666667: 100%|██████████| 235/235 [00:27<00:00,  8.44it/s]



Test set: Average loss: 0.0177, Accuracy: 9943/10000 (99.430%)

